# get timezones

In [1]:
from datetime import datetime, timedelta
import pytz
import numpy as np

import plotly.express as px
import pandas as pd

In [2]:
list(pytz.all_timezones_set)[:10]

['Asia/Istanbul',
 'Asia/Katmandu',
 'MST7MDT',
 'Africa/Timbuktu',
 'America/Indiana/Knox',
 'Canada/Saskatchewan',
 'America/Rainy_River',
 'America/Tortola',
 'America/Knox_IN',
 'Asia/Jayapura']

In [7]:
def inputs_to_rowdicts(user_timezone, input_name, input_timezone, input_hourrange):
    
    start_hour, end_hour = input_hourrange[0], input_hourrange[1]

    user_now = datetime.now(tz=pytz.timezone(user_timezone))
    
    input_now = user_now.astimezone(pytz.timezone(input_timezone))

    input_start = input_now - timedelta(hours = 24)

    times = pd.date_range(input_start, periods=72, freq='1h').tolist()
    
    #keep obs if they are the times of interest
    time_blips = [time for time in times if time.hour == start_hour or time.hour == end_hour]
    
    # fill in missing start/end times
    if time_blips[0].hour == end_hour:
        time_blips.insert(0, np.min(times))
    if time_blips[-1].hour == start_hour:
        time_blips.insert(-1, np.max(times))
    
    # convert into users timezone
    user_timeblips = [time.astimezone(pytz.timezone(user_timezone)) for time in time_blips]
    
    # create dictionary of rows for the inputs
    row_dict_list = []
    for x in range(int(len(user_timeblips)/2)):
              row_dict_list.append({'person':input_name,'start':user_timeblips.pop(0),'end':user_timeblips.pop(0)})
        
    return row_dict_list
    


In [8]:
def input_to_dataframe(inputs):

    timezone_inputs = inputs[::2]

    hourrange_inputs = inputs[1::2]         

    name_inputs = ['You'] + [f"Person {str(i + 1)}" for i in range(int(len(inputs)/2 - 1))]

    user_tz = timezone_inputs[0] 

    all_waketimes = []                 
    for person in zip(name_inputs, timezone_inputs, hourrange_inputs):

        rowdict = inputs_to_rowdicts(user_timezone = user_tz, 
                                    input_name = person[0],
                                    input_timezone = person[1], 
                                    input_hourrange = person[2])

        all_waketimes = all_waketimes + rowdict

    return pd.DataFrame(all_waketimes)

# get hours

In [ ]:
import pandas as pd

date_today = '1996-02-12 08:00:00'

times = pd.date_range(date_today, periods=24, freq='1h')


times


# create graph

## create label mapping dictionary

In [11]:
ylabs = ['12am'] + [f"{str(x)}am" for x in range(1, 12)] +  ['12pm'] + [f"{str(x)}pm" for x in range(1, 12)]

lab_dict = {k: v for k, v in zip(np.arange(0, 24), ylabs)}

In [ ]:
lab_dict

In [141]:
date_today = '1996-02-12'

times = pd.date_range(date_today, periods=24, freq='1h')

times



DatetimeIndex(['1996-02-12 00:00:00', '1996-02-12 01:00:00',
               '1996-02-12 02:00:00', '1996-02-12 03:00:00',
               '1996-02-12 04:00:00', '1996-02-12 05:00:00',
               '1996-02-12 06:00:00', '1996-02-12 07:00:00',
               '1996-02-12 08:00:00', '1996-02-12 09:00:00',
               '1996-02-12 10:00:00', '1996-02-12 11:00:00',
               '1996-02-12 12:00:00', '1996-02-12 13:00:00',
               '1996-02-12 14:00:00', '1996-02-12 15:00:00',
               '1996-02-12 16:00:00', '1996-02-12 17:00:00',
               '1996-02-12 18:00:00', '1996-02-12 19:00:00',
               '1996-02-12 20:00:00', '1996-02-12 21:00:00',
               '1996-02-12 22:00:00', '1996-02-12 23:00:00'],
              dtype='datetime64[ns]', freq='H')

In [128]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame([
    dict(Task="You", Start='1996-02-12 08:00:00', Finish='1996-02-12 17:00:00'),
    dict(Task="Person 1", Start='1996-02-12 08:00:00', Finish = '1996-02-12 17:00:00')
])

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()


In [184]:
inputs = ['Australia/Victoria',
          [8, 20],
          'Europe/Zurich',
          [6, 20],
          'NZ',
          [8,20]]

In [204]:
df = input_to_dataframe(inputs)
waketimes_df = find_waketimes(df, 'Australia/Victoria')

In [206]:
fig = px.timeline(df, x_start="start", x_end="end", y="person")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
for idx, row in waketimes_df.iterrows():
    fig.add_vrect(x0=row['start'], x1=row['end'])
fig.show()

# getting limits

In [16]:
def get_limits(user_timezone):

    user_now = datetime.now(tz=pytz.timezone(user_timezone))

    user_start = user_now - timedelta(hours=2)

    times = pd.date_range(user_start, periods=26, freq='1h').tolist()

    return np.min(times), np.max(times)

In [17]:
get_limits("NZ")

(Timestamp('2021-10-28 22:19:47.768511+1300', tz='NZ', freq='H'),
 Timestamp('2021-10-29 23:19:47.768511+1300', tz='NZ', freq='H'))

# getting date overlap

# get hourly obs

In [211]:
def round_to_hour(time):
    return (time.replace(second=0, microsecond=0, minute=0, hour=time.hour)
               +timedelta(hours=time.minute//30))

def get_times(user_timezone):
    
    user_now = datetime.now(tz=pytz.timezone(user_timezone))
    
    user_start = round_to_hour(user_now) - timedelta(hours=2)

    times = pd.date_range(user_start, periods=26, freq='1h').tolist()
    
    return times


In [ ]:
get_times('Australia/Victoria')

In [55]:
def find_match(df, user_timezone): 

    user_now = datetime.now(tz=pytz.timezone(user_timezone))

    user_start = user_now - timedelta(hours=2)

    times = pd.date_range(user_start, periods=26, freq='1h')
    
    time_dict = {}
    for time in times:
        n_wakes = 0
        for idx, row in df.iterrows():
            if time >= row['start'] and time <= row['end']:
                n_wakes += 1
        time_dict[time] = n_wakes
    
    matches = {k: v for k, v in time_dict.items() if v == len(df.person.unique())}.keys() 
    
    return matches #np.min(matches), np.max(matches)

# Agglomerative clustering

In [ ]:
X = np.array([match for match in find_match(df, 'Australia/Victoria')]).reshape(-1, 1)

#X = np.vstack([X, np.array(x[-1] + np.timedelta64(2, 'h'))])

X_values = np.array([[x[0].value] for x in X])
X_values

from sklearn.cluster import AgglomerativeClustering

agg_clusts = AgglomerativeClustering(n_clusters = None , linkage = 'single', distance_threshold = 3600000000001).fit(X_values)

labs = agg_clusts.labels_

intervals = [list(X.reshape(-1)[labs == lab]) for lab in np.unique(labs)]

intervals
    

In [201]:
def old_find_waketimes(df, user_timezone): 
    x = [match for match in find_match(df, user_timezone)]

    timeslots_list = []
    for i in range(len(x)-1):
        if ((x[i+1] - x[i]) > np.timedelta64(1, 'h')): # or (i+1 == len(x)-1):
            timeslots_list.append(x[:i+1])
            x = x[i+1:]

    intervals = timeslots_list + [x]

    return pd.DataFrame([{'start':pd.to_datetime(np.min(interval)), 'end': pd.to_datetime(np.max(interval))} for interval in intervals])


In [ ]:
old_find_waketimes(df, 'Australia/Victoria')